In [1]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB

# define parameters
periods = 12

demand = [66, 78, 57, 34, 5, 118, 76, 61, 23, 31, 68, 48]
forecast = 50

initial_inv = 0

setupcost = 450
holdingcost = 2
backordercost = 20
safetystock = 30
Big = sum(demand[t] for t in range(periods))

quantity = np.zeros(periods)

smoothing = 0.3 #exponential smoothing

In [2]:
#define function to call with each horizon option
def rolling_horizon(): #add more arguments for more flexibility, e.g. safetystock = None, smoothing constant, etc.
    #in case of using/changing initial_inv and forecast as global variable; otherwise must add to function arguments
    global initial_inv
    global forecast
    
    #define run horizon
    runs = periods-horizon+1
    
    #rolling over run horizon
    for k in range(runs): 
        m = gp.Model()
        lotsize = {}
        inventory ={}
        setup = {}
        
        # define decision variables
        for t in range(horizon):
            lotsize[t] = m.addVar()
            inventory[t] = m.addVar(obj=holdingcost)
            setup[t] = m.addVar(vtype=GRB.BINARY, obj=setupcost)
        
        # create constraints
        # initial inventory
        m.addConstr(inventory[0] == initial_inv+lotsize[0]-forecast)  
        
        # inventory balance
        for t in range(1,horizon):
            m.addConstr(inventory[t-1]+lotsize[t]-forecast == inventory[t])
        
        # logic constraint between lotsize and setup
        for t in range(horizon):
            m.addConstr(lotsize[t] <= Big*setup[t])

        # safety stock
        for t in range(horizon):
            m.addConstr(inventory[t] >= safetystock)
            
        # run model
        m.optimize()
        
        # update after each run: lotsize, starting inventory, forecast
        quantity[k] = lotsize[0].x #order of this period
        initial_inv = initial_inv + lotsize[0].x - demand[k] #inventory at the end of period after demand realization
        forecast = smoothing*demand[k] + (1-smoothing)*forecast #update forecast for next run
        m.dispose
        
    
    #repeat forecast for out-of-run period 
    for k in range(horizon-1):
        quantity[runs+k] = lotsize[k+1].x  


    # print result
    print("\n\nSolution:\n")
    print(quantity)
    
    cost = 0
    inv = 0
    for t in range(periods):
        if quantity[t] > 0:
            cost = cost+setupcost
            inv = inv+quantity[t]-demand[t]
        if inv > 0:
            cost=cost+holdingcost*inv
        else: cost=cost-backordercost*inv  
        
    print("Rolling Horizon Cost: "+str(cost)) 


In [4]:
horizon = 4
rolling_horizon()

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 12 rows, 12 columns and 23 nonzeros
Model fingerprint: 0xc9a070b0
Variable types: 8 continuous, 4 integer (4 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+02]
  Objective range  [2e+00, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+01, 5e+01]
Presolve removed 7 rows and 4 columns
Presolve time: 0.00s
Presolved: 5 rows, 8 columns, 12 nonzeros
Variable types: 5 continuous, 3 integer (3 binary)

Root relaxation: objective 1.109143e+03, 3 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1109.14313    0    1          - 1109.14313      -     -    0s
H    0     0                    1453.1799808 1109.14313  23.7%     -    0s
H    0     0                    1316.3599616 1109.14313  15.7%     -    0s

Cutting planes:
  Flow cover: 1

Explored 1 nod

Thread count was 8 (of 8 available processors)

Solution count 3: 1389.6 1438.8 1514.4 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.389601245041e+03, best bound 1.389601245041e+03, gap 0.0000%
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 12 rows, 12 columns and 23 nonzeros
Model fingerprint: 0xef050c58
Variable types: 8 continuous, 4 integer (4 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+02]
  Objective range  [2e+00, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 7e+01]
Presolve removed 7 rows and 4 columns
Presolve time: 0.00s
Presolved: 5 rows, 8 columns, 12 nonzeros
Variable types: 5 continuous, 3 integer (3 binary)

Root relaxation: objective 1.223841e+03, 3 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1223.84115    0    1          - 1223.84115      -

In [8]:
horizon = 6
rolling_horizon()

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 18 rows, 18 columns and 35 nonzeros
Model fingerprint: 0x8599969e
Variable types: 12 continuous, 6 integer (6 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+02]
  Objective range  [2e+00, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+01, 5e+01]
Presolve removed 9 rows and 4 columns
Presolve time: 0.02s
Presolved: 9 rows, 14 columns, 22 nonzeros
Variable types: 9 continuous, 5 integer (5 binary)

Root relaxation: objective 1.300617e+03, 5 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1300.61701    0    3          - 1300.61701      -     -    0s
H    0     0                    2473.7118406 1300.61701  47.4%     -    0s
H    0     0                    1991.9942947 1300.61701  34.7%     -    0s
H    0     0                    1887.4236811 1


    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  824.53738    0    2          -  824.53738      -     -    0s
H    0     0                    1630.5576038  824.53738  49.4%     -    0s
H    0     0                    1412.0782328  824.53738  41.6%     -    0s
     0     0     cutoff    0      1412.07823 1412.07823  0.00%     -    0s

Cutting planes:
  Gomory: 1
  Implied bound: 2
  MIR: 1
  Flow cover: 3

Explored 1 nodes (5 simplex iterations) in 0.02 seconds
Thread count was 8 (of 8 available processors)

Solution count 2: 1412.08 1630.56 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.412078232808e+03, best bound 1.412078232808e+03, gap 0.0000%
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 18 rows, 18 columns and 35 nonzeros
Model fingerprint: 0xfe7bd183
Variable types: 12 continuous, 6 integer (6 binary)
Coefficient stati

In [5]:
horizon = 8
rolling_horizon()

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 24 rows, 24 columns and 47 nonzeros
Model fingerprint: 0x0e146a94
Variable types: 16 continuous, 8 integer (8 binary)
Coefficient statistics:
  Matrix range     [1e+00, 7e+02]
  Objective range  [2e+00, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+01, 5e+01]
Presolve removed 11 rows and 4 columns
Presolve time: 0.00s
Presolved: 13 rows, 20 columns, 32 nonzeros
Variable types: 13 continuous, 7 integer (7 binary)

Root relaxation: objective 1.491379e+03, 7 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1491.37908    0    5          - 1491.37908      -     -    0s
H    0     0                    3493.7118406 1491.37908  57.3%     -    0s
H    0     0                    3053.1298164 1491.37908  51.2%     -    0s
H    0     0                    2802.853067


Cutting planes:
  Implied bound: 5
  MIR: 1
  Flow cover: 4
  Flow path: 2

Explored 1 nodes (15 simplex iterations) in 0.03 seconds
Thread count was 8 (of 8 available processors)

Solution count 5: 2295.28 2401.24 2533.35 ... 2877.39

Optimal solution found (tolerance 1.00e-04)
Best objective 2.295276935555e+03, best bound 2.295276935555e+03, gap 0.0000%


Solution:

[ 55.61017776 130.62853068   0.         149.14843646   0.
   0.         115.55793011   0.           0.         158.94330646
   0.           0.        ]
Rolling Horizon Cost: 6702.074886715781
